环境准备
%pip install datasets langchain sentence_transformers tqdm chromadb langchain_wenxin

In [1]:
# 本地数据加载
from langchain.document_loaders import TextLoader
loader = TextLoader("C:\\Users\\administered\\ai\\deepLearning\\li.txt", encoding="utf-8")
documents = loader.load()
documents

[Document(metadata={'source': 'C:\\Users\\administered\\ai\\deepLearning\\li.txt'}, page_content='藜（读音lí）麦（Chenopodium\\xa0quinoa\\xa0Willd.）是藜科藜属植物。穗部可呈红、紫、黄，植株形状类似灰灰菜，成熟后穗部类似高粱穗。植株大小受环境及遗传因素影响较大，从0.3-3米不等，茎部质地较硬，可分枝可不分。单叶互生，叶片呈鸭掌状，叶缘分为全缘型与锯齿缘型。藜麦花两性，花序呈伞状、穗状、圆锥状，藜麦种子较小，呈小圆药片状，直径1.5-2毫米，千粒重1.4-3克。\\xa0[1]\\xa0\\n原产于南美洲安第斯山脉的哥伦比亚、厄瓜多尔、秘鲁等中高海拔山区。具有一定的耐旱、耐寒、耐盐性，生长范围约为海平面到海拔4500米左右的高原上，最适的高度为海拔3000-4000米的高原或山地地区。\\xa0[1]\\xa0\\n藜麦富含的维生素、多酚、类黄酮类、皂苷和植物甾醇类物质具有多种健康功效。')]

In [3]:
# 文档分割
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 128, chunk_overlap = 0) # 采用固定字符长度分割chunk_size=128
documents = text_splitter.split_documents(documents)
documents
print(len(documents[0].page_content))  # 输出文本的字符数

359
